# Read Data

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


## Required Imports

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
path="/content/drive/MyDrive/ML_project_datasheets/Final_code/labeled_data.csv"
data=pd.read_csv(path)

In [ ]:
data.head()

,Unnamed: 0,reviewText,Review_without_stopwords,After_lemmatization,pos_tags,senti_score,Overall Sentiment
0,3878097,I thought it was depressing,thought depressing,thought depressing,"[('thought', 'NN'), ('depressing', 'VBG')]",0.125,Positive
1,1390262,One of my all time favorite flicks. An adult t...,one time favorite flicks adult themed comedy d...,one time favorite flick adult themed comedy dr...,"[('one', 'CD'), ('time', 'NN'), ('favorite', '...",1.375,Positive
2,4629692,LOVE. LOVE. LOVE. Great movie.,love love love great movie,love love love great movie,"[('love', 'VB'), ('love', 'NN'), ('love', 'NN'...",1.250,Positive
3,6433712,The product was deliver excellent and just on ...,product deliver excellent time,product deliver excellent time,"[('product', 'NN'), ('deliver', 'VB'), ('excel...",1.000,Positive
4,4837513,Terrific movie.,terrific movie,terrific movie,"[('terrific', 'JJ'), ('movie', 'NN')]",0.000,Neutral


In [ ]:
data['reviews_text_new'] = data['After_lemmatization'].copy()

In [ ]:
data.shape

(500000, 7)

In [ ]:
data.head()

,Unnamed: 0,reviewText,Review_without_stopwords,After_lemmatization,pos_tags,senti_score,Overall Sentiment,reviews_text_new
0,3878097,I thought it was depressing,thought depressing,thought depressing,"[('thought', 'NN'), ('depressing', 'VBG')]",0.125,Positive,thought depressing
1,1390262,One of my all time favorite flicks. An adult t...,one time favorite flicks adult themed comedy d...,one time favorite flick adult themed comedy dr...,"[('one', 'CD'), ('time', 'NN'), ('favorite', '...",1.375,Positive,one time favorite flick adult themed comedy dr...
2,4629692,LOVE. LOVE. LOVE. Great movie.,love love love great movie,love love love great movie,"[('love', 'VB'), ('love', 'NN'), ('love', 'NN'...",1.250,Positive,love love love great movie
3,6433712,The product was deliver excellent and just on ...,product deliver excellent time,product deliver excellent time,"[('product', 'NN'), ('deliver', 'VB'), ('excel...",1.000,Positive,product deliver excellent time
4,4837513,Terrific movie.,terrific movie,terrific movie,"[('terrific', 'JJ'), ('movie', 'NN')]",0.000,Neutral,terrific movie


In [ ]:
Data = data[['reviews_text_new', 'Overall Sentiment']]

In [ ]:
Data['Overall Sentiment'].head()

0    Positive
1    Positive
2    Positive
3    Positive
4     Neutral
Name: Overall Sentiment, dtype: object

In [ ]:
Data.head(10)

,reviews_text_new,Overall Sentiment
0,thought depressing,Positive
1,one time favorite flick adult themed comedy dr...,Positive
2,love love love great movie,Positive
3,product deliver excellent time,Positive
4,terrific movie,Neutral
5,great addictive,Negative
6,fun qwerky,Positive
7,year old son obsessed rented redbox cried retu...,Positive
8,forget hurry literate scurrying read play noth...,Positive
9,watching like looking future see thing happeni...,Negative


## Encode lables

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(Data['Overall Sentiment'])

In [ ]:
labels[0:10]

array([2, 2, 2, 2, 1, 0, 2, 2, 2, 0])

#  Feature Extraction using CountVectorizer.

In [ ]:
# Create a CountVectorizer to convert text to a bag-of-words matrix
bow_counts = CountVectorizer()
bow_data = bow_counts.fit_transform(Data['reviews_text_new'])

In [ ]:
bow_data.shape

(500000, 198145)

This indicates that in the first document (row 0), the word or n-gram at index 176109 appears once, and the word or n-gram at index 49847 appears once.

In [ ]:
print(bow_data[0])

  (0, 176109)	1
  (0, 49847)	1


# Preprocess data so we can feed it to LSTM

In [ ]:
# Convert the bag-of-words matrix to sequences of integers
sequences = [list(doc.nonzero()[1]) for doc in bow_data]

In [ ]:
for seq in sequences[0]:
    print(seq)

176109
49847


In [ ]:
max_sequence_length=100

In [ ]:
# Pad the sequences for equal length
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

In [ ]:
X = padded_sequences
y = labels

## LSTM(3 layers) without balancing

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
embedding_dim = 50
lstm_units = 64
output_units=3

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=bow_data.shape[1], output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(units=lstm_units))
model.add(Dense(units=output_units, activation='softmax'))


sparse_categorical_crossentropy: This loss function is suitable when your labels are integer-encoded. In this case, each label is just an integer indicating the class index.

In [ ]:
# Compile and train the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=32)

Epoch 1/10
10938/10938 [==============================] - 1483s 135ms/step - loss: 0.3042 - accuracy: 0.8848
Epoch 2/10
10938/10938 [==============================] - 1475s 135ms/step - loss: 0.2182 - accuracy: 0.9180
Epoch 3/10
10938/10938 [==============================] - 1438s 131ms/step - loss: 0.1586 - accuracy: 0.9413
Epoch 4/10
10938/10938 [==============================] - 1415s 129ms/step - loss: 0.1226 - accuracy: 0.9542
Epoch 5/10
10938/10938 [==============================] - 1390s 127ms/step - loss: 0.0984 - accuracy: 0.9635
Epoch 6/10
10938/10938 [==============================] - 1386s 127ms/step - loss: 0.0800 - accuracy: 0.9704
Epoch 7/10
10938/10938 [==============================] - 1402s 128ms/step - loss: 0.0662 - accuracy: 0.9758
Epoch 8/10
10938/10938 [==============================] - 1398s 128ms/step - loss: 0.0558 - accuracy: 0.9797
Epoch 9/10
10938/10938 [==============================] - 1432s 131ms/step - loss: 0.0466 - accuracy: 0.9832
Epoch 10/10
10938/1

In [ ]:
from sklearn.metrics import classification_report

y_pred = model.predict(X_test)
# print(classification_report(y_test,y_pred))

y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_test, y_pred_classes))


4688/4688 [==============================] - 65s 14ms/step
              precision    recall  f1-score   support

           0       0.67      0.64      0.65     21651
           1       0.73      0.82      0.78     28033
           2       0.91      0.88      0.90    100316

    accuracy                           0.84    150000
   macro avg       0.77      0.78      0.78    150000
weighted avg       0.84      0.84      0.84    150000



## LSTM (3 layers) using Random Under Sampler

In [ ]:
rus = RandomUnderSampler(random_state=42)
X_rus, y_rus = rus.fit_resample(X, y)

In [ ]:
X_train_rus,X_test_rus,y_train_rus,y_test_rus=train_test_split(X_rus, y_rus,test_size=0.2, random_state=0)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=bow_data.shape[1], output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(units=lstm_units))
model.add(Dense(units=output_units, activation='softmax'))

In [ ]:
# Compile and train the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train_rus, y_train_rus, epochs=10, batch_size=32)

Epoch 1/10
5431/5431 [==============================] - 740s 136ms/step - loss: 0.4350 - accuracy: 0.8257
Epoch 2/10
5431/5431 [==============================] - 751s 138ms/step - loss: 0.2796 - accuracy: 0.8946
Epoch 3/10
5431/5431 [==============================] - 764s 141ms/step - loss: 0.1915 - accuracy: 0.9283
Epoch 4/10
5431/5431 [==============================] - 751s 138ms/step - loss: 0.1412 - accuracy: 0.9468
Epoch 5/10
5431/5431 [==============================] - 730s 134ms/step - loss: 0.1095 - accuracy: 0.9591
Epoch 6/10
5431/5431 [==============================] - 733s 135ms/step - loss: 0.0874 - accuracy: 0.9673
Epoch 7/10
5431/5431 [==============================] - 743s 137ms/step - loss: 0.0704 - accuracy: 0.9738
Epoch 8/10
5431/5431 [==============================] - 761s 140ms/step - loss: 0.0573 - accuracy: 0.9791
Epoch 9/10
5431/5431 [==============================] - 749s 138ms/step - loss: 0.0464 - accuracy: 0.9832
Epoch 10/10
5431/5431 [=======================

In [ ]:
y_pred = model.predict(X_test_rus)
# print(classification_report(y_test,y_pred))

y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_test_rus, y_pred_classes))

1358/1358 [==============================] - 23s 16ms/step
              precision    recall  f1-score   support

           0       0.83      0.76      0.79     14419
           1       0.84      0.85      0.85     14578
           2       0.79      0.85      0.82     14447

    accuracy                           0.82     43444
   macro avg       0.82      0.82      0.82     43444
weighted avg       0.82      0.82      0.82     43444



## LSTM (3 layers) using Random Over sampler

In [ ]:
ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(X, y)

In [ ]:
X_train_ros,X_test_ros,y_train_ros,y_test_ros=train_test_split(X_ros, y_ros,test_size=0.2, random_state=0)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=bow_data.shape[1], output_dim=embedding_dim, input_length=max_sequence_length))
model.add(LSTM(units=lstm_units))
model.add(Dense(units=output_units, activation='softmax'))

In [ ]:
# Compile and train the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train_ros, y_train_ros, epochs=10, batch_size=32)

Epoch 1/10
25065/25065 [==============================] - 7188s 287ms/step - loss: 0.2922 - accuracy: 0.8888
Epoch 2/10
25065/25065 [==============================] - 7188s 287ms/step - loss: 0.1494 - accuracy: 0.9462
Epoch 3/10
25065/25065 [==============================] - 7112s 284ms/step - loss: 0.0988 - accuracy: 0.9654
Epoch 4/10
25065/25065 [==============================] - 6986s 279ms/step - loss: 0.0703 - accuracy: 0.9758
Epoch 5/10
25065/25065 [==============================] - 7088s 283ms/step - loss: 0.0519 - accuracy: 0.9826
Epoch 6/10
25065/25065 [==============================] - 7199s 287ms/step - loss: 0.0401 - accuracy: 0.9866
Epoch 7/10
25065/25065 [==============================] - 7048s 281ms/step - loss: 0.0319 - accuracy: 0.9894
Epoch 8/10
25065/25065 [==============================] - 7107s 284ms/step - loss: 0.0263 - accuracy: 0.9913
Epoch 9/10
25065/25065 [==============================] - 7285s 291ms/step - loss: 0.0225 - accuracy: 0.9925
Epoch 10/10
24520/2

In [ ]:
y_pred = model.predict(X_test_ros)
# print(classification_report(y_test,y_pred))

y_pred_classes = np.argmax(y_pred, axis=1)
print(classification_report(y_test_ros, y_pred_classes))

6267/6267 [==============================] - 116s 19ms/step
              precision    recall  f1-score   support

           0       0.97      0.99      0.98     66906
           1       0.97      0.98      0.97     66948
           2       0.97      0.94      0.96     66664

    accuracy                           0.97    200518
   macro avg       0.97      0.97      0.97    200518
weighted avg       0.97      0.97      0.97    200518



# GradientBoosting

In [ ]:
X = padded_sequences
y = labels

## Gradient Boosting without balncing

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [ ]:
X_train.shape

(450000, 100)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
# Define model with differant parameters.
gbc= GradientBoostingClassifier(n_estimators=100,learning_rate=0.1,max_depth=2,warm_start=True,random_state=0)

In [ ]:
# Fit trainig and testing data to model
gbc.fit(X_train, y_train)

GradientBoostingClassifier(max_depth=2, random_state=0, warm_start=True)

In [ ]:
# predict using testing data
y_pred_gb =gbc.predict(X_test)

In [ ]:
# Model evaluation
accuracy = accuracy_score(y_test, y_pred_gb)
print(f"Accuracy: {accuracy:.2f}")

# Classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(y_test, y_pred_gb))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred_gb))

Accuracy: 0.74

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.01      0.01      7175
           1       0.77      0.47      0.59      9327
           2       0.74      0.97      0.84     33498

    accuracy                           0.74     50000
   macro avg       0.75      0.48      0.48     50000
weighted avg       0.74      0.74      0.67     50000


Confusion Matrix:
[[   39   356  6780]
 [    0  4429  4898]
 [   14   988 32496]]


## Gradient Boosting with random over sampling

In [ ]:
X = padded_sequences
y = labels

In [ ]:
ros = RandomOverSampler(random_state=42)
X_ros, y_ros = ros.fit_resample(X, y)

In [ ]:
X_train_ros,X_test_ros,y_train_ros,y_test_ros=train_test_split(X_ros, y_ros,test_size=0.1, random_state=0)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
# Define model with differant parameters.
gbc= GradientBoostingClassifier(n_estimators=100,learning_rate=0.1,max_depth=2,warm_start=True,random_state=0)


In [ ]:
# Fit trainig and testing data to model
gbc.fit(X_train_ros, y_train_ros)

GradientBoostingClassifier(max_depth=2, random_state=0, warm_start=True)

In [ ]:
# predict using testing data
y_pred_gb =gbc.predict(X_test_ros)

In [ ]:
# Model evaluation
accuracy = accuracy_score(y_test_ros, y_pred_gb)
print(f"Accuracy: {accuracy:.2f}")

# Classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(y_test_ros, y_pred_gb))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test_ros, y_pred_gb))

Accuracy: 0.56

Classification Report:
              precision    recall  f1-score   support

           0       0.48      0.68      0.56     33432
           1       0.67      0.78      0.72     33499
           2       0.54      0.23      0.33     33328

    accuracy                           0.56    100259
   macro avg       0.57      0.56      0.54    100259
weighted avg       0.57      0.56      0.54    100259


Confusion Matrix:
[[22643  5957  4832]
 [ 5613 26176  1710]
 [18721  6843  7764]]


## Gradient Boosting with random under sampling

In [ ]:
X = padded_sequences
y = labels

In [ ]:
rus = RandomUnderSampler(random_state=42)
X_rus, y_rus = rus.fit_resample(X, y)
X_train_rus,X_test_rus,y_train_rus,y_test_rus=train_test_split(X_rus, y_rus,test_size=0.1, random_state=0)

In [ ]:
gbc= GradientBoostingClassifier(n_estimators=100,learning_rate=0.01,max_depth=2,warm_start=True,random_state=0)

In [ ]:
gbc.fit(X_train_rus, y_train_rus)

GradientBoostingClassifier(learning_rate=0.01, max_depth=2, random_state=0,
                           warm_start=True)

In [ ]:
y_pred_gb =gbc.predict(X_test_rus)

In [ ]:
# Model evaluation
accuracy = accuracy_score(y_test_rus, y_pred_gb)
print(f"Accuracy: {accuracy:.2f}")

# Classification report and confusion matrix
print("\nClassification Report:")
print(classification_report(y_test_rus, y_pred_gb))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test_rus, y_pred_gb))

Accuracy: 0.53

Classification Report:
              precision    recall  f1-score   support

           0       0.46      0.79      0.58     14419
           1       0.65      0.76      0.70     14578
           2       0.38      0.04      0.07     14447

    accuracy                           0.53     43444
   macro avg       0.50      0.53      0.45     43444
weighted avg       0.50      0.53      0.45     43444


Confusion Matrix:
[[11359  2419   641]
 [ 3279 11008   291]
 [10316  3553   578]]
